#### Implementing early stopping

In [1]:
import os
import re
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Flatten, InputLayer
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import utils
import keras
import imageio 
from PIL import Image 

In [3]:
# Reading the data
train = pd.read_csv('agedetectiontrain/train.csv')

# Image resizing of test data into single numpy array
temp = []
for img_name in train.ID:
    img_path = os.path.join('agedetectiontrain/Train', img_name)
    img = imageio.imread(img_path)
    img = np.array(Image.fromarray(img).resize((32, 32))).astype('float32')    
    temp.append(img)

train_x = np.stack(temp)

# Normalizing the images
train_x = train_x / 255.

# Encoding the categorical variable to numer
lb = LabelEncoder()
train_y = lb.fit_transform(train.Class)
train_y = keras.utils.to_categorical(train_y)

C:\Users\anaswara\AppData\Local\Temp\ipykernel_1256\2433241514.py:8: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread(img_path)


In [4]:
# Specifying all the parameters we will be using in our network
input_num_units = (32, 32, 3)
hidden_num_units = 500
output_num_units = 3

epochs = 100
batch_size = 512

In [6]:
model = Sequential([
          InputLayer(input_shape=input_num_units),
          Flatten(),
          Dense(units=hidden_num_units, activation='relu'),
          Dense(units=output_num_units, activation='softmax'),
        ])

# Defining parameters like optmizer, loss function and evaluating metric
model.compile(loss='categorical_crossentropy', # 
        optimizer=keras.optimizers.Adam(), # Learning rate and momentum can be passed inside optimizer
        metrics=['accuracy'])

# Defining early stopping callback.
# After epoch 50, training can be stopped if not improved in 'val_loss' is seen

cb_early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=8, min_delta=0.02) 
cb = keras.callbacks.TensorBoard(log_dir='early_stop', write_graph=False) 

history = model.fit(train_x, train_y, epochs=epochs, validation_split=0.2, callbacks=[cb, cb_early_stop])

Epoch 1/100
498/498 ━━━━━━━━━━━━━━━━━━━━ 20s 36ms/step - accuracy: 0.5605 - loss: 1.0705 - val_accuracy: 0.5899 - val_loss: 0.8662
Epoch 2/100
498/498 ━━━━━━━━━━━━━━━━━━━━ 16s 32ms/step - accuracy: 0.6103 - loss: 0.8324 - val_accuracy: 0.6261 - val_loss: 0.8006
Epoch 3/100
498/498 ━━━━━━━━━━━━━━━━━━━━ 16s 32ms/step - accuracy: 0.6192 - loss: 0.8141 - val_accuracy: 0.6336 - val_loss: 0.7892
Epoch 4/100
498/498 ━━━━━━━━━━━━━━━━━━━━ 18s 36ms/step - accuracy: 0.6325 - loss: 0.7971 - val_accuracy: 0.6185 - val_loss: 0.8519
Epoch 5/100
498/498 ━━━━━━━━━━━━━━━━━━━━ 18s 36ms/step - accuracy: 0.6328 - loss: 0.7910 - val_accuracy: 0.6570 - val_loss: 0.7690
Epoch 6/100
498/498 ━━━━━━━━━━━━━━━━━━━━ 17s 28ms/step - accuracy: 0.6488 - loss: 0.7734 - val_accuracy: 0.6650 - val_loss: 0.7545
Epoch 7/100
498/498 ━━━━━━━━━━━━━━━━━━━━ 16s 33ms/step - accuracy: 0.6548 - loss: 0.7569 - val_accuracy: 0.6447 - val_loss: 0.7704
Epoch 8/100
498/498 ━━━━━━━━━━━━━━━━━━━━ 17s 33ms/step - accuracy: 0.6607 - loss: 0

#### Training stopped on epoch 21 due to `val_loss` not improving from min_delta = 0.02